In [ ]:
#CONEXÃO GOOGLE DRIVE
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install pandas

#Dimensões e Fato

In [ ]:
import pandas as pd
import numpy as np

ano = 2015
pasta_dados = '/content/drive/MyDrive/FACULDADE 2025/TCC/DADOS V3/Dados_Finais' #ALTERAR PASTA

while ano <= 2024:
  print(ano)
  df_temp = pd.read_csv(f'{pasta_dados}/FinalX{ano}.csv',encoding='utf-8',sep=';',low_memory=False)
  if ano == 2015:
    df = df_temp
  else:
    df = pd.concat([df, df_temp], axis=0)
  del df_temp
  ano +=1
print('pronto')

2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
pronto


In [ ]:
#display(df)

In [ ]:
#Local
dim_local = df[['sigla-uf','UF','Municipio','Regiao']].drop_duplicates()
dim_local['municipio_codigo'] = dim_local.groupby('UF')['Municipio'].rank(method='dense').astype(int)
dim_local['Cod_Local'] = dim_local['sigla-uf'] + '-' + dim_local['municipio_codigo'].astype(str)
dim_local.drop(columns='municipio_codigo', inplace=True)
dim_local= dim_local[['Cod_Local','Regiao','UF','sigla-uf','Municipio']]
dim_local = dim_local.sort_values(by=['Regiao']).reset_index(drop=True)
dim_local['Cod_Local'] = 'L' + '-' + dim_local['Cod_Local']
dim_local.to_csv('/content/drive/MyDrive/FACULDADE 2025/TCC/DADOS V3/Fato_e_Dimensões/dim_local.csv',index=False,sep=';', encoding='utf-8')

In [ ]:
#Cbo
dim_cbo = df[['Familia CBO 2002','Cargo CBO 2002','Cod CBO 2002']].drop_duplicates()
dim_cbo = dim_cbo.sort_values(by=['Cargo CBO 2002']).reset_index(drop=True)
dim_cbo['Cod_CBO_2002'] = 'C' + '-'+ dim_cbo['Cod CBO 2002']
dim_cbo= dim_cbo[['Cod_CBO_2002','Cargo CBO 2002','Familia CBO 2002']]
dim_cbo.to_csv('/content/drive/MyDrive/FACULDADE 2025/TCC/DADOS V3/Fato_e_Dimensões/dim_cbo.csv',index=False,sep=';', encoding='utf-8')

In [ ]:
#Tempo
dim_tempo = df[['Ano','Periodo']].drop_duplicates()
mapa_meses = {
    'Janeiro': 1,'Fevereiro': 2,'Março': 3,'Abril': 4,
    'Maio': 5,'Junho': 6,'Julho': 7,'Agosto': 8,'Setembro': 9,
    'Outubro': 10,'Novembro': 11,'Dezembro': 12,'Não admitido ano':13,'Não desligado ano':14
}
dim_tempo['Numero_Mes'] = dim_tempo ['Periodo'].map(mapa_meses).fillna(15).astype(int)
dim_tempo['Cod_Ano'] = dim_tempo['Ano'].astype(str) + '-' + dim_tempo['Numero_Mes'].astype(str).str.zfill(2)
dim_tempo= dim_tempo[['Cod_Ano','Ano','Periodo','Numero_Mes']]
dim_tempo = dim_tempo.sort_values(by=['Ano', 'Numero_Mes']).reset_index(drop=True)
dim_tempo['Cod_Ano'] = 'A' + '-'+ dim_tempo['Cod_Ano']
dim_tempo.to_csv('/content/drive/MyDrive/FACULDADE 2025/TCC/DADOS V3/Fato_e_Dimensões/dim_tempo.csv',index=False,sep=';', encoding='utf-8')

In [ ]:
#Pessoa
dim_pessoa = df[['Sexo','Faixa Etaria','Status','Tempo_Emprego','Media_Remu']].drop_duplicates()
num_linhas = len(dim_pessoa)
num_digitos = len(str(num_linhas))
dim_pessoa = dim_pessoa.reset_index(drop=True)
dim_pessoa['Cod_Pessoa'] = dim_pessoa.index + 1
dim_pessoa['Cod_Pessoa'] = dim_pessoa['Cod_Pessoa'].astype(str).str.zfill(num_digitos)
dim_pessoa['Cod_Pessoa'] = 'P'+ '-' + dim_pessoa['Cod_Pessoa']
dim_pessoa = dim_pessoa[['Cod_Pessoa','Sexo','Status','Faixa Etaria','Tempo_Emprego','Media_Remu']]
dim_pessoa.to_csv('/content/drive/MyDrive/FACULDADE 2025/TCC/DADOS V3/Fato_e_Dimensões/dim_pessoa.csv',index=False,sep=';', encoding='utf-8')

In [ ]:
#foi feito assim devido a quantidade de dados o suporte nao suporta tanto processamento ao mesmo tempo
ano = 2015
while ano <= 2024 :
  df_temp = df[df['Ano'] == ano]
  df_temp = df_temp.merge(dim_tempo,how='left',on=['Periodo','Ano'])
  df_temp = df_temp.merge(dim_local,how='left',on=['sigla-uf','UF','Municipio','Regiao'])
  df_temp = df_temp.merge(dim_pessoa,how='left',on=['Sexo','Faixa Etaria','Status','Tempo_Emprego','Media_Remu'])
  df_temp = df_temp.merge(dim_cbo,how='left',on=['Cargo CBO 2002','Familia CBO 2002'])
  df_temp.to_csv(f'/content/drive/MyDrive/FACULDADE 2025/TCC/DADOS V3/Fato_e_Dimensões/Dados_Tratados{ano}.csv',index=False,sep=';', encoding='utf-8') #ALTERAR PASTA
  del df_temp
  print(ano)
  ano += 1

2015
2016
2017
2018
2019
2020
2021
2022
2023
2024


In [ ]:
#fato
#foi feito assim devido a quantidade de dados o suporte nao suporta tanto processamento ao mesmo tempo
import pandas as pd
a = 2015
while a <= 2024 :
  df_temp = pd.read_csv(f'/content/drive/MyDrive/FACULDADE 2025/TCC/DADOS V3/Fato_e_Dimensões/Dados_Tratados{a}.csv' ,sep=';', encoding='utf-8',low_memory=False) #ALTERAR PASTA
  fato_temp = df_temp.groupby(['Cod_Ano', 'Cod_Pessoa', 'Cod_CBO_2002', 'Cod_Local']).agg(
      TotalAdmitidos=('Status', lambda x: (x == 'Admitido').sum()),
      TotalDemitidos=('Status', lambda x: (x == 'Demitido').sum()),
      RegistrosAnuais=('Status', lambda x: (x == 'Ativo').sum()),
      PermanenciasAnuais=('Status', lambda x: (x == 'Nao demitidos').sum()),
      RegistrosAnosAnterior=('Status', lambda x: (x == 'Admitidos ano anterior').sum()),
      TotalFaixaEtaria=('Faixa Etaria', 'count'),
      TotalFaixaMediaRemuneracao=('Media_Remu', 'count'),
      TotalFaixaTempoEmprego=('Tempo_Emprego', 'count'),
      TotalSexo=('Sexo', 'count')
  ).reset_index()
  if a == 2015:
    fato = fato_temp
  else:
    fato = pd.concat([fato,fato_temp],axis=0, ignore_index=True)
  del df_temp
  del fato_temp
  print(a)
  a += 1
print('salvando')
fato.to_csv('/content/drive/MyDrive/FACULDADE 2025/TCC/DADOS V3/Fato_e_Dimensões/fato.csv',index=False,sep=';', encoding='utf-8')
print('salvo')

2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
salvando
salvo
